In [2]:
# Optional: run to install the libraries locally if you haven't already 
!pip3 install langchain
!pip3 install openai
!pip3 install neo4j

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import json 
import pandas as pd

In [6]:

# Optional: run to load environment variables from a .env file.
# This is not required if you have exported your env variables in another way or if you set it manually
!pip3 install python-dotenv
from dotenv import load_dotenv
load_dotenv()

# Set the OpenAI API key env variable manually
# os.environ["OPENAI_API_KEY"] = "<your_api_key>"

# print(os.environ["OPENAI_API_KEY"])

Defaulting to user installation because normal site-packages is not writeable


True

In [9]:
# Loading a json dataset from a file
file_path = 'data/testindatajson.json'

with open(file_path, 'r') as file:
    jsonData = json.load(file)

In [10]:
df =  pd.read_json(file_path)
df.head()

,product_id,product,relationship,entity_type,entity_value,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,Blackout Curtain,hasCategory,category,home decoration,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
1,1925202,Blackout Curtain,hasBrand,brand,ArtzFolio,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
2,1925202,Blackout Curtain,hasCharacteristic,characteristic,Eyelets,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
3,1925202,Blackout Curtain,hasCharacteristic,characteristic,Tie Back,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
4,1925202,Blackout Curtain,hasCharacteristic,characteristic,100% opaque,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98


In [11]:
# DB credentials
url = "neo4j+s://15b0eec3.databases.neo4j.io"
username ="neo4j"
password = "rQSQYXSfcmZT3MSrpYd6ba7owTk2YGFrlxckWSWptFs"

In [14]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [15]:
def sanitize(text):
    text = str(text).replace("'","").replace('"','').replace('{','').replace('}', '')
    return text

# Loop through each JSON object and add them to the db
i = 1
for obj in jsonData:
    print(f"{i}. {obj['product_id']} -{obj['relationship']}-> {obj['entity_value']}")
    i+=1
    query = f'''
        MERGE (product:Product {{id: {obj['product_id']}}})
        ON CREATE SET product.name = "{sanitize(obj['product'])}", 
                       product.title = "{sanitize(obj['TITLE'])}", 
                       product.bullet_points = "{sanitize(obj['BULLET_POINTS'])}", 
                       product.size = {sanitize(obj['PRODUCT_LENGTH'])}

        MERGE (entity:{obj['entity_type']} {{value: "{sanitize(obj['entity_value'])}"}})

        MERGE (product)-[:{obj['relationship']}]->(entity)
        '''
    graph.query(query)

1. 1925202 -hasCategory-> home decoration
2. 1925202 -hasBrand-> ArtzFolio
3. 1925202 -hasCharacteristic-> Eyelets
4. 1925202 -hasCharacteristic-> Tie Back
5. 1925202 -hasCharacteristic-> 100% opaque
6. 1925202 -hasCharacteristic-> Heavy premium cotton canvas fabric
7. 1925202 -hasMeasurement-> Width 4.5 feet (54 inch) Height 5 feet (60 inch)
8. 1925202 -hasColor-> Multicolour
9. 1925202 -isFor-> adults
10. 2673191 -hasCategory-> clothing
11. 2673191 -hasBrand-> Marks & Spencer
12. 2673191 -hasColor-> Navy Mix
13. 2673191 -hasCharacteristic-> 100% cotton
14. 2673191 -hasCharacteristic-> Ribbing
15. 2673191 -hasMeasurement-> 9-10Y
16. 2673191 -isFor-> children
17. 2765088 -hasCategory-> automotive accessories
18. 2765088 -hasCharacteristic-> Loud Dual Tone Trumpet
19. 2765088 -hasCharacteristic-> High Quality
20. 2765088 -hasCharacteristic-> High Intensity Horn Sound
21. 2765088 -hasCharacteristic-> Professional installation required
22. 2765088 -hasColor-> Red
23. 2765088 -hasMeasureme

In [16]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"


In [17]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model=embeddings_model),
    url=url,
    username=username,
    password=password,
    index_name='products',
    node_label="Product",
    text_node_properties=['name', 'title'],
    embedding_node_property='embedding',
)

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [18]:
def embed_entities(entity_type):
    vector_index = Neo4jVector.from_existing_graph(
        OpenAIEmbeddings(model=embeddings_model),
        url=url,
        username=username,
        password=password,
        index_name=entity_type,
        node_label=entity_type,
        text_node_properties=['value'],
        embedding_node_property='embedding',
    )
    
entities_list = df['entity_type'].unique()

for t in entities_list:
    embed_entities(t)

In [19]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [20]:
chain.run("""
Help me find curtains
""")

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Category {name: "Curtains"})-[:BELONGS_TO]->(p:Product)
RETURN p;
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [23]:
entity_types = {
    "product": "Item detailed type, for example 'high waist pants', 'outdoor plant pot', 'chef kitchen knife'",
    "category": "Item category, for example 'home decoration', 'women clothing', 'office supply'",
    "characteristic": "if present, item characteristics, for example 'waterproof', 'adhesive', 'easy to use'",
    "measurement": "if present, dimensions of the item", 
    "brand": "if present, brand of the item",
    "color": "if present, color of the item",
    "age_group": "target age group for the product, one of 'babies', 'children', 'teenagers', 'adults'. If suitable for multiple age groups, pick the oldest (latter in the list)."
}

relation_types = {
    "hasCategory": "item is of this category",
    "hasCharacteristic": "item has this characteristic",
    "hasMeasurement": "item is of this measurement",
    "hasBrand": "item is of this brand",
    "hasColor": "item is of this color", 
    "isFor": "item is for this age_group"
 }

entity_relationship_match = {
    "category": "hasCategory",
    "characteristic": "hasCharacteristic",
    "measurement": "hasMeasurement", 
    "brand": "hasBrand",
    "color": "hasColor",
    "age_group": "isFor"
}

In [24]:
system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database. 
    
    The graph database links products to the following entity types:
    {json.dumps(entity_types)}
    
    Each link has one of the following relationships:
    {json.dumps(relation_types)}

    Depending on the user prompt, determine if it possible to answer with the graph database.
        
    The graph database can match products with multiple relationships to several entities.
    
    Example user input:
    "Which blue clothing items are suitable for adults?"
    
    There are three relationships to analyse:
    1. The mention of the blue color means we will search for a color similar to "blue"
    2. The mention of the clothing items means we will search for a category similar to "clothing"
    3. The mention of adults means we will search for an age_group similar to "adults"
    
    
    Return a json object following the following rules:
    For each relationship to analyse, add a key value pair with the key being an exact match for one of the entity types provided, and the value being the value relevant to the user query.
    
    For the example provided, the expected output would be:
    {{
        "color": "blue",
        "category": "clothing",
        "age_group": "adults"
    }}
    
    If there are no relevant entities in the user prompt, return an empty json object.
'''

print(system_prompt)


    You are a helpful agent designed to fetch information from a graph database. 
    
    The graph database links products to the following entity types:
    {"product": "Item detailed type, for example 'high waist pants', 'outdoor plant pot', 'chef kitchen knife'", "category": "Item category, for example 'home decoration', 'women clothing', 'office supply'", "characteristic": "if present, item characteristics, for example 'waterproof', 'adhesive', 'easy to use'", "measurement": "if present, dimensions of the item", "brand": "if present, brand of the item", "color": "if present, color of the item", "age_group": "target age group for the product, one of 'babies', 'children', 'teenagers', 'adults'. If suitable for multiple age groups, pick the oldest (latter in the list)."}
    
    Each link has one of the following relationships:
    {"hasCategory": "item is of this category", "hasCharacteristic": "item has this characteristic", "hasMeasurement": "item is of this measurement", "hasB

In [25]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

# Define the entities to look for
def define_query(prompt, model="gpt-4-1106-preview"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format= {
            "type": "json_object"
        },
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion.choices[0].message.content

In [26]:
example_queries = [
    "Which pink items are suitable for children?",
    "Help me find gardening gear that is waterproof",
    "I'm looking for a bench with dimensions 100x50 for my living room"
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")

Q: 'Which pink items are suitable for children?'
{
    "color": "pink",
    "age_group": "children"
}

Q: 'Help me find gardening gear that is waterproof'
{
    "category": "gardening gear",
    "characteristic": "waterproof"
}

Q: 'I'm looking for a bench with dimensions 100x50 for my living room'
{
    "product": "bench",
    "measurement": "100x50",
    "category": "living room"
}



In [27]:
def create_embedding(text):
    result = client.embeddings.create(model=embeddings_model, input=text)
    return result.data[0].embedding

In [28]:
# The threshold defines how closely related words should be. Adjust the threshold to return more or less results
def create_query(text, threshold=0.81):
    query_data = json.loads(text)
    # Creating embeddings
    embeddings_data = []
    for key, val in query_data.items():
        if key != 'product':
            embeddings_data.append(f"${key}Embedding AS {key}Embedding")
    query = "WITH " + ",\n".join(e for e in embeddings_data)
    # Matching products to each entity
    query += "\nMATCH (p:Product)\nMATCH "
    match_data = []
    for key, val in query_data.items():
        if key != 'product':
            relationship = entity_relationship_match[key]
            match_data.append(f"(p)-[:{relationship}]->({key}Var:{key})")
    query += ",\n".join(e for e in match_data)
    similarity_data = []
    for key, val in query_data.items():
        if key != 'product':
            similarity_data.append(f"gds.similarity.cosine({key}Var.embedding, ${key}Embedding) > {threshold}")
    query += "\nWHERE "
    query += " AND ".join(e for e in similarity_data)
    query += "\nRETURN p"
    return query

In [29]:
def query_graph(response):
    embeddingsParams = {}
    query = create_query(response)
    query_data = json.loads(response)
    for key, val in query_data.items():
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    result = graph.query(query, params=embeddingsParams)
    return result

In [33]:
example_response = '''{
    "category": "clothes",
    "color": "blue",
    "age_group": "adults"
}'''

result = query_graph(example_response)

ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'gds.similarity.cosine' (line 8, column 7 (offset: 271))
"WHERE gds.similarity.cosine(categoryVar.embedding, $categoryEmbedding) > 0.81 AND gds.similarity.cosine(colorVar.embedding, $colorEmbedding) > 0.81 AND gds.similarity.cosine(age_groupVar.embedding, $age_groupEmbedding) > 0.81"
       ^}

In [34]:
# Result
print(f"Found {len(result)} matching product(s):\n")
for r in result:
    print(f"{r['p']['name']} ({r['p']['id']})")

NameError: name 'result' is not defined

In [35]:
# Adjust the relationships_threshold to return products that have more or less relationships in common
def query_similar_items(product_id, relationships_threshold = 3):
    
    similar_items = []
        
    # Fetching items in the same category with at least 1 other entity in common
    query_category = '''
            MATCH (p:Product {id: $product_id})-[:hasCategory]->(c:category)
            MATCH (p)-->(entity)
            WHERE NOT entity:category
            MATCH (n:Product)-[:hasCategory]->(c)
            MATCH (n)-->(commonEntity)
            WHERE commonEntity = entity AND p.id <> n.id
            RETURN DISTINCT n;
        '''
    

    result_category = graph.query(query_category, params={"product_id": int(product_id)})
    #print(f"{len(result_category)} similar items of the same category were found.")
          
    # Fetching items with at least n (= relationships_threshold) entities in common
    query_common_entities = '''
        MATCH (p:Product {id: $product_id})-->(entity),
            (n:Product)-->(entity)
            WHERE p.id <> n.id
            WITH n, COUNT(DISTINCT entity) AS commonEntities
            WHERE commonEntities >= $threshold
            RETURN n;
        '''
    result_common_entities = graph.query(query_common_entities, params={"product_id": int(product_id), "threshold": relationships_threshold})
    #print(f"{len(result_common_entities)} items with at least {relationships_threshold} things in common were found.")

    for i in result_category:
        similar_items.append({
            "id": i['n']['id'],
            "name": i['n']['name']
        })
            
    for i in result_common_entities:
        result_id = i['n']['id']
        if not any(item['id'] == result_id for item in similar_items):
            similar_items.append({
                "id": result_id,
                "name": i['n']['name']
            })
    return similar_items

In [36]:
product_ids = ['1519827', '2763742']

for product_id in product_ids:
    print(f"Similar items for product #{product_id}:\n")
    result = query_similar_items(product_id)
    print("\n")
    for r in result:
        print(f"{r['name']} ({r['id']})")
    print("\n\n")

Similar items for product #1519827:



Womens Cotton Ankle Length Leggings (1594019)
A Line Open Back Satin Prom Dress (1955999)
Turtleneck Oversized Sweaters (2535064)
Womens Satin Semi-Stitched Lehenga Choli (2763742)
Womens Drawstring Harem Pants (1233616)
Underwire Bra (1325580)
Chiffon printed dupatta (2919319)
Womens Lightly Padded Non-Wired Printed T-Shirt Bra (1954205)
Glossies Thong (941830)
Sun Uv Protection Driving Gloves (1844637)
Dhoti Pants (2293307)
Womens Maroon Round Neck Full Sleeves Gathered Peplum Top (1256928)
V-Neck Long Jumpsuit (2838428)
Alpine Faux Suede Knit Pencil Skirt (1372443)
Lingerie for Women for Sex Naughty (2666747)
Maxi Dresses (1818763)
Womens Shift Knee-Long Dress (1483279)



Similar items for product #2763742:



Womens Cotton Ankle Length Leggings (1594019)
A Line Open Back Satin Prom Dress (1955999)
Turtleneck Oversized Sweaters (2535064)
Plain V Neck Half Sleeves T Shirt (1519827)
Womens Drawstring Harem Pants (1233616)
Underwire Bra (1325580)

In [37]:
def query_db(params):
    matches = []
    # Querying the db
    result = query_graph(params)
    for r in result:
        product_id = r['p']['id']
        matches.append({
            "id": product_id,
            "name":r['p']['name']
        })
    return matches  

In [38]:
def similarity_search(prompt, threshold=0.8):
    matches = []
    embedding = create_embedding(prompt)
    query = '''
            WITH $embedding AS inputEmbedding
            MATCH (p:Product)
            WHERE gds.similarity.cosine(inputEmbedding, p.embedding) > $threshold
            RETURN p
            '''
    result = graph.query(query, params={'embedding': embedding, 'threshold': threshold})
    for r in result:
        product_id = r['p']['id']
        matches.append({
            "id": product_id,
            "name":r['p']['name']
        })
    return matches

In [39]:
prompt_similarity = "I'm looking for nice curtains"
print(similarity_search(prompt_similarity))

ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'gds.similarity.cosine' (line 4, column 19 (offset: 95))
"            WHERE gds.similarity.cosine(inputEmbedding, p.embedding) > $threshold"
                   ^}